In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append("/home/nick/C2S-Python-API")
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely

In [ ]:
import noaa_coops

In [ ]:
# sample bbox from a geojson
bbox = [[
    [
      -75.01920876517276,
      36.528583600142895
    ],
    [
      -80.17839668640761,
      36.528583600142895
    ],
    [
      -80.17839668640761,
      32.4610893038282
    ],
    [
      -75.01920876517276,
      32.4610893038282
    ],
    [
      -75.01920876517276,
      36.528583600142895
    ]
  ]
]
bbox = bbox[0]
bbox

In [ ]:
lats = min([x[1] for x in bbox]), max([x[1] for x in bbox])
lons = min([x[0] for x in bbox]), max([x[0] for x in bbox])

stations = noaa_coops.get_stations_from_bbox(lat_coords=lats, lon_coords=lons)
stations

In [ ]:
test_station = noaa_coops.Station(stations[0])
test_station.name

In [ ]:
df_water_levels = test_station.get_data(
    begin_date="20210101",
    end_date="20210201",
    product="water_level",
    datum="MLLW",
    units="metric",
)

In [ ]:
df_water_levels

[Description of what the columns represent](https://api.tidesandcurrents.noaa.gov/api/prod/responseHelp.html)

In [ ]:
df_water_levels.plot(y="v")

In [ ]:
test_station.products

Full list of options for `"product"`:

```
[
    "water_level",
    "hourly_height",
    "high_low",
    "daily_mean",
    "monthly_mean",
    "one_minute_water_level",
    "predictions",
    "datums",
    "air_gap",
    "air_temperature",
    "water_temperature",
    "wind",
    "air_pressure",
    "conductivity",
    "visibility",
    "humidity",
    "salinity",
    "currents",
    "currents_predictions",
    "ofs_water_level",
]
```

There may be some others available that are considered "meteorlogical conditions".

## Checking hourly levels for any NaNs or other strange values

Look at the record for Sandy Hook (according to an [NYT article](https://www.nytimes.com/2014/01/14/science/tide-gauges-needed-for-research-are-often-victims-of-storms.html), it washed out during Hurricane Sandy on October 29, 2012).

`An astonishing 73 tide stations were damaged or destroyed as Sandy swept across the North Atlantic basin, including ones in Puerto Rico and the United States Virgin Islands.`

In [ ]:
station = noaa_coops.Station("8531680")

In [ ]:
print(station.name)

In [ ]:
# check Sandy Hook gauge during Hurricane Sandy
df_hourly = station.get_data(
    begin_date="20121028",
    end_date="20121030",
    product="hourly_height",
    datum="MLLW",
    units="metric",
)
print(len(df_hourly))

In [ ]:
# add a few more days and see if it's the same length
df_hourly = station.get_data(
    begin_date="20121028",
    end_date="20121105",
    product="hourly_height",
    datum="MLLW",
    units="metric",
)
print(len(df_hourly))

In [ ]:
df_hourly

In [ ]:
# get the expected hours
expected_hours = pd.date_range("2012-10-28", "2012-11-05", freq="H")
print(len(expected_hours))

In [ ]:
# find missing hours
[x for x in expected_hours if x not in df_hourly.index]

In [ ]:
# test: get years of data for a gauge, check for missing data
start_date = "2010-01-01"
end_date = "2020-01-01"
df_water_levels = test_station.get_data(
    begin_date=start_date.replace("-", ""),
    end_date=end_date.replace("-", ""),
    product="hourly_height",
    datum="MSL",
    units="metric",
)

expected_hours = pd.date_range(start_date, end_date, freq="H")

print(len(df_water_levels))
print(len(expected_hours))

In [ ]:
# if the timeframe encompasses a no-data period, the no-data period is filled with NaNs
# so either missing rows or NaNs indicate outages
def search_for_outages(station_id, start_date, end_date, debug=False):
    """Return (bool, df) where bool is True if dates are missing
    start_date and end_date in "YYYY-mm-dd" format.
    """
    station = noaa_coops.Station(station_id)
    df = station.get_data(
        begin_date=start_date.replace("-", ""),
        end_date=end_date.replace("-", ""),
        product="hourly_height",
        datum="MSL",
        units="metric",
    )

    # missing rows are one indication of outages
    # but need to account for starting availability
    record_start = station.data_inventory["Verified Hourly Height Water Level"]["start_date"].split(" ")[0]
    record_end = station.data_inventory["Verified Hourly Height Water Level"]["end_date"].split(" ")[0]
    expected_hours = pd.date_range(
        max(start_date, record_start), min(end_date, record_end), freq="H"
    )
    # if the record start/end date is different than submitted, print as an alert
    if debug:
        if (start_date != max(start_date, record_start)) or (end_date != min(end_date, record_end)):
            print(f"start: {max(start_date, record_start)}")
            print(f"end:   {min(end_date, record_end)}")
    num_missing = len(expected_hours) - len(df)

    # NaNs are another indication
    nan_hours = np.sum(df["v"].isna())
    
    # flags set to (1,1) are the last indication
    flagged_hours = np.sum(df["f"].apply(lambda x: x=="1,1"))
    
    if any(x > 0 for x in [nan_hours, flagged_hours]):
        if debug:
            print(f"{nan_hours=}")
            print(f"{flagged_hours=}")
        print(f"{station_id} ({station.name}) is missing {nan_hours} hours.")
    else:
        print(f"{station_id} ({station.name}) has no missing hours.")        

    return nan_hours, df

In [ ]:
# use on a known case (Sandy Hook, NJ during a period covering Hurricane Sandy)
missing, df = search_for_outages("8531680", "2010-01-01", "2015-01-01")
missing

In [ ]:
df.plot(y="v")

In [ ]:
# some atlantic stations
stations = [
    '8651370',
    '8652587',
    '8654467',
    '8656483',
    '8658120',
    '8658163',
    '8661070',
    '8665530'
]

missing_hours_counts = []
dfs = {}
for station_id in stations:
    missing, df = search_for_outages(station_id, "2000-01-01", "2020-01-01")
    missing_hours_counts.append(missing)
    dfs[station_id] = df
    print("#"*10)

In [ ]:
# check on the Charleston station where there are 16 flagged hours but with apparently valid data
df = dfs["8665530"]
df.loc[df["f"] == "1,1"]

In [ ]:
df.loc["2015-02-20", :].plot(y="v")

In [ ]:
# what's going on with Springmaid Pier, where we see a massive outage?
dfs["8661070"].plot(y="v")
plt.show()
dfs["8661070"].loc["2000-11-30":"2000-12-31", :].plot(y="v")
plt.show()

In [ ]:
# what's going on with Springmaid Pier, where we see a massive outage?
missing, df = search_for_outages("8661070", "2000-01-01", "2020-01-01", debug=True)

In [ ]:
df.loc[df["v"].isna(), :]

## Checking results when querying for very long records

In [ ]:
station = noaa_coops.Station("8661070")
print(station.name)

In [ ]:
station.data_inventory

In [ ]:
# hourly water level starts in 1976. What happens if we query before then?
out = station.get_data(
    begin_date="19700101",
    end_date="20240101",
    product="hourly_height",
    datum="MSL",
)
print(len(out))

In [ ]:
out.head()

## Merging dataframes with different lengths
- Keep same index `t`
- Keep v/s/f fields, but add an extra index indicating the station number

In [ ]:
df1 = dfs[stations[0]]
df2 = dfs[stations[2]]
print(len(df1), len(df2))

In [ ]:
merged_df = pd.concat([df1, df2], keys=[stations[0], stations[2]], axis=1)
merged_df

In [ ]:
merged_df[stations[2]]

In [ ]:
fig, ax = plt.subplots(1,1)
merged_df[stations[0]].plot(ax=ax, y="v", label=stations[0])
merged_df[stations[2]].plot(ax=ax, y="v", label=stations[2])
ax.set_ylabel("gauge height (MSL)")
plt.show()

# Get full data record for all Atlantic stations

In [ ]:
# can I use a nice geojson instead of just a bbox?
# no. bbox it is, then.
big_geom = [
            [
              -65.66547220355395,
              47.26526924937144
            ],
            [
              -70.50291308804213,
              45.08503647353169
            ],
            [
              -73.55751503025883,
              43.23221741177247
            ],
            [
              -78.62254525972577,
              39.07613426999106
            ],
            [
              -81.58254149491296,
              34.24771068739118
            ],
            [
              -82.91077131974842,
              30.999542363635413
            ],
            [
              -80.64070727870458,
              25.998811021385663
            ],
            [
              -80.4967240501288,
              24.898305587208228
            ],
            [
              -79.13777976049207,
              26.50457433121548
            ],
            [
              -78.6817855079541,
              30.891384212027106
            ],
            [
              -74.22090082631496,
              35.331890706458864
            ],
            [
              -73.21492407841056,
              38.45502499546171
            ],
            [
              -68.4983608573623,
              41.254452775746785
            ],
            [
              -66.52988282288521,
              43.238472154019945
            ],
            [
              -65.66547220355395,
              47.26526924937144
            ]
          ]

In [ ]:
# get ids for all stations in this giant geometry
lats = min([x[1] for x in big_geom]), max([x[1] for x in big_geom])
lons = min([x[0] for x in big_geom]), max([x[0] for x in big_geom])

stations = noaa_coops.get_stations_from_bbox(lat_coords=lats, lon_coords=lons)
print(len(stations))

In [ ]:
# get records for all and merge into one big dataframe that we can save
# search for any outages along the way
missing_hours_counts = []
dfs = {}
for station_id in stations:
    try:
        missing, df = search_for_outages(station_id, "1900-01-01", "2024-01-01", debug=True)
        missing_hours_counts.append(missing)
        dfs[station_id] = df
    except Exception as e:
        print(f"{station_id}: {e}")
    print("#"*10)

In [ ]:
# make a big dataframe and pickle it
merged_df = pd.concat(list(dfs.values()), keys=list(dfs.keys()), axis=1)
merged_df

In [ ]:
# pickle this massive guy
merged_df.to_pickle("/data/surge/tide-gauge-hourly-records-eastcoast.pkl")

In [ ]:
merged_df.memory_usage().sum() / 1e3 / 1000

In [ ]:
# remove flags
idx = pd.IndexSlice
merged_df = merged_df.loc[:, idx[:, ["v", "s"]]]
# make sparse
merged_df = merged_df.astype(pd.SparseDtype("float", np.nan))

In [ ]:
merged_df.memory_usage().sum() / 1e3 / 1000

In [ ]:
# pickle the sparse version
merged_df.to_pickle("/data/surge/tide-gauge-hourly-records-eastcoast-sparse.pkl")

## How many gauges are completely continuous?
Answer: none! but that may not be as bad as it seems; it doesn't mean this was always due to storms creating outages

In [ ]:
missing_hours_counts

In [ ]:
dfs["8656483"].plot(y="v")

In [ ]:
noaa_coops.Station("8656483").name

In [ ]:
# number of no-data since 2000?
np.sum(dfs["8656483"].loc["2000-01-01":, "v"].isna())

In [ ]:
# what about for a different gauge?
np.sum(dfs[stations[2]].loc["2000-01-01":, "v"].isna())

In [ ]:
np.sum(merged_df[stations[2]].loc["2000-01-01":, "v"].isna())

In [ ]:
dfs[stations[2]].loc["2000-01-01":, "v"]

In [ ]:
merged_df[stations[2]].loc["2000-01-01":, "v"]

## What is our average start date?

In [ ]:
merged_df = pd.read_pickle("/data/surge/tide-gauge-hourly-records-eastcoast-sparse.pkl")

In [ ]:
from tenacity import retry

In [ ]:
@retry
def get_record_start(station_id):
    return noaa_coops.Station(station_id).data_inventory["Verified Hourly Height Water Level"]["start_date"].split(" ")[0]

def print_record_start(station_id):
    print(get_record_start(station_id))

record_start_dates = []
for station_id in stations:
    record_start_dates.append(get_record_start(station_id))

In [ ]:
record_start_dates

## (jff) look at annual mean through time

In [ ]:
idx = pd.IndexSlice
years = pd.date_range("1930-01-01", "2023-01-01", freq="Y")
out = []
for year in years:
    out.append(np.nanmean(merged_df.loc[str(year.year), idx[:, "v"]]))

plt.plot([year.year for year in years], out)
plt.ylabel("MSL (national tidal datum) (m)")
plt.title("Annual average of NOAA tidal gauges")
plt.show()

# Spatial properties of available tide gauges

In [ ]:
merged_df = pd.read_pickle("/data/surge/tide-gauge-hourly-records-eastcoast-sparse.pkl")

## Plot on a map

In [ ]:
stations = [x[0] for x in merged_df.columns]
stations_info = {}

for station_id in stations:
    station = noaa_coops.Station(station_id)
    stations_info[station_id] = {}
    stations_info[station_id]["name"] = station.name
    stations_info[station_id]["lat"] = station.lat_lon["lat"]
    stations_info[station_id]["lon"] = station.lat_lon["lon"]

In [ ]:
df = pd.DataFrame.from_dict(stations_info).T
df

In [ ]:
df.loc[:, "geometry"] = df.apply(lambda x: shapely.geometry.Point(x["lon"], x["lat"]), axis=1)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)
gdf

In [ ]:
borders = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.set_xlim(-85, -65)
ax.set_ylim(25, 46)
# put a simple coastline as a background
borders.plot(ax=ax)
# add tide gauges
gdf.plot(color="red", ax=ax, label="NOAA gauges")
plt.legend()
plt.show()

## Use DB of US cities to see how many cities are near a tide gauge

In [ ]:
cities = pd.read_csv("/data/static/uscities.csv")
cities.loc[:, "geometry"] = cities.apply(lambda x: shapely.geometry.Point(x["lng"], x["lat"]), axis=1)
cities_gdf = gpd.GeoDataFrame(cities, geometry="geometry", crs=4326)

In [ ]:
cities_gdf.loc[cities_gdf["population"] > 100000]

In [ ]:
borders = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.set_xlim(-85, -65)
ax.set_ylim(25, 46)
# put a simple coastline as a background
borders.plot(ax=ax)

# add tide gauges
gdf.plot(color="red", ax=ax, label="NOAA gauges")

# add cities with a population over 100,000
cities_gdf.loc[cities_gdf["population"] > 100000].plot(color='yellow', ax=ax, label="cities w pop>100k")

# plot
plt.legend()
plt.show()

In [ ]:
# buffer tide gauges by 25km
# USGS contiguous US albers equal area
epsg = 5070


borders = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
borders = borders.loc[borders["name"].isin(["Canada", "United States of America"])]

fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.set_xlim(1.25*1e6, 2.25*1e6)
ax.set_ylim(0.25*1e6, 3.25*1e6)

# put a simple coastline as a background
borders.to_crs(epsg).plot(ax=ax, alpha=0.5)

# add tide gauges
gdf.to_crs(epsg).buffer(25000).plot(color="red", ax=ax, label="NOAA gauges w 25km buffer")

# add cities with a population over 100,000
cities_gdf.to_crs(epsg).loc[cities_gdf["population"] > 100000].plot(color='yellow', ax=ax, label="cities w pop>100k")

# plot
plt.legend()
plt.show()

In [ ]:
# do some geometric "isin" operations
pop_minimum = 100000


# cities within 25km of a tide gauge
out = cities_gdf[cities_gdf["population"] > pop_minimum].to_crs(epsg)["geometry"].within(gdf.to_crs(epsg).buffer(25000).unary_union)
print(f"Cities within 25km of a tide gauge: {np.sum(out)}")

# within 15km
out = cities_gdf[cities_gdf["population"] > pop_minimum].to_crs(epsg)["geometry"].within(gdf.to_crs(epsg).buffer(15000).unary_union)
print(f"Cities within 15km of a tide gauge: {np.sum(out)}")

# 10km
out = cities_gdf[cities_gdf["population"] > pop_minimum].to_crs(epsg)["geometry"].within(gdf.to_crs(epsg).buffer(10000).unary_union)
print(f"Cities within 10km of a tide gauge: {np.sum(out)}")

# 5km
out = cities_gdf[cities_gdf["population"] > pop_minimum].to_crs(epsg)["geometry"].within(gdf.to_crs(epsg).buffer(5000).unary_union)
print(f"Cities within 5km of a tide gauge: {np.sum(out)}")


In [ ]:
# print names of cities with population greater than pop_minimum within 5km of a tide gauge
cities_gdf.loc[out & (cities_gdf["population"] > pop_minimum), ["city", "state_id"]].apply(lambda x: (x["city"], x["state_id"]), axis=1).tolist()

In [ ]:
# buffer tide gauges by 25km
# USGS contiguous US albers equal area
epsg = 5070


borders = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
borders = borders.loc[borders["name"].isin(["Canada", "United States of America"])]

fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.set_xlim(0.50*1e6, 2.5*1e6)
ax.set_ylim(0.25*1e6, 3.25*1e6)

# put a simple coastline as a background
borders.to_crs(epsg).plot(ax=ax, alpha=0.5)

# add tide gauges
# gdf.to_crs(epsg).buffer(25000).plot(color="red", ax=ax, label="NOAA gauges w 25km buffer")

# add cities with a population over 100,000
cities_gdf.loc[out & (cities_gdf["population"] > pop_minimum), :].to_crs(epsg).plot(
    color='green', 
    ax=ax, 
    label="cities w pop>100k within 5km of gauges",
)

# plot
plt.legend()
plt.show()

In [ ]:
pop_minimum = 100000
buffer_radius = 8000  # meters

out = cities_gdf[cities_gdf["population"] > pop_minimum].to_crs(epsg)["geometry"].within(gdf.to_crs(epsg).buffer(buffer_radius).unary_union)
print(f"Cities within {buffer_radius/1000}km of a tide gauge: {np.sum(out)}")
cities_gdf.loc[out & (cities_gdf["population"] > pop_minimum), ["city", "state_id"]].apply(lambda x: (x["city"], x["state_id"]), axis=1).tolist()

In [ ]:
# are the NYC stations in this dataset?
# 8518750 (The Battery) and 8516945 (Kings Point)
print("8518750" in gdf.index)
print("8516945" in gdf.index)
print("8519483" in gdf.index)

# Temporal properties

## How continuous are the NaNs in the records?

In [ ]:
merged_df = pd.read_pickle("/data/surge/tide-gauge-hourly-records-eastcoast-sparse.pkl")

### Pick an arbitrary record for testing
`"8518750"`

In [ ]:
# get the indices of the valid records
valid_hours = [i for i, x in enumerate(merged_df["8518750"]["v"].isna()) if not x]

In [ ]:
# get the index of first valid hour
print(valid_hours[0])
# create a series starting here
series = merged_df["8518750"].iloc[valid_hours[0]:valid_hours[-1]]

In [ ]:
series

In [ ]:
# check how many NaNs remain
np.sum(series["v"].isna())

In [ ]:
# this feels like an interview problem
invalid_hours = [i for i, x in enumerate(series["v"].isna()) if x]
invalid_hours[:100]

In [ ]:
invalid_hours[-1]

In [ ]:
len(series)

In [ ]:
series.iloc[-1]

In [ ]:
start = 0
end = 0

for i, x in enumerate(series["v"].isna()):
    if x:
        # if we're at a NaN index, start counting
        start = i
        break

# start at the NaN index, look for the point where it becomes valid
for i, x in enumerate(series["v"].iloc[start:].isna()):
    if not x:
        # when we reach a valid index, stop counting
        end = (i + start)
        break

consecutive_nans = end - start
print(consecutive_nans)

In [ ]:
# wrap previous cell into a function
def consecutive_nans(series):
    start = 0
    end = 0
    running = True
    nan_periods = []

    while running:
        for i, x in enumerate(series.iloc[start:].isna()):
            if x:
                # if we're at a NaN index, start counting
                start = i
                break
        
        # start at the NaN index, look for the point where it becomes valid
        for i, x in enumerate(series.iloc[start:].isna()):
            if not x:
                # when we reach a valid index, stop counting
                end = (i + start)
                break

        consecutive_nans = end - start
        if consecutive_nans == 0:
            break
    
        start = end
        print(consecutive_nans)
        nan_periods.append(consecutive_nans)
        
    return nan_periods

In [ ]:
out = consecutive_nans(series["v"])

In [ ]:
out

In [ ]:
print(start)
print(end)

In [ ]:
np.sum(series["v"].iloc[56712:].isna())

so there are still NaNs even after that big chunk?

In [ ]:
# this feels like an interview problem
invalid_hours = [i for i, x in enumerate(series["v"].iloc[51700:].isna()) if x]
invalid_hours[:100]

In [ ]:
# https://stackoverflow.com/questions/41968892/counting-consecutive-numbers-in-a-list
def countlist(l):
    count = 0
    retlist = []
    # Avoid IndexError for  random_list[i+1]
    for i in range(len(l) - 1):
        # Check if the next number is consecutive
        if l[i] + 1 == l[i+1]:
            count += 1
        else:
            # If it is not append the count and restart counting
            retlist.append(count)
            count = 1
    # Since we stopped the loop one early append the last count
    retlist.append(count)
    return retlist

In [ ]:
# counts of the number of consecutive NaNs
out = countlist(invalid_hours)
out

In [ ]:
plt.hist(out)
plt.xlabel("consecutive hours of missing data")
plt.ylabel('counts')
plt.show()

## Making this a little cleaner and more efficient

In [ ]:
# get the indices of the valid records
series = merged_df["8518750"]["v"]
valid_hours = [i for i, x in enumerate(merged_df["8518750"]["v"].isna()) if not x]

# cut off any NaNs at the beginning or end
series = series.iloc[valid_hours[0]:valid_hours[-1]]

In [ ]:
invalid_hours = [i for i, x in enumerate(series.isna()) if x]

In [ ]:
consecutive_nans = countlist(invalid_hours)

In [ ]:
plt.hist(np.array(consecutive_nans) / 24, bins=30)
plt.xlabel("consecutive days of missing data")
plt.ylabel('counts')
plt.title(f"Gauge 8518750")
plt.show()

## Repeating for all gauges

In [ ]:
from tqdm import tqdm

In [ ]:
stations = [x[0] for x in merged_df.columns.tolist()]
stations = list(set(stations))
consecutive_nan_dict = {}
for station_id in tqdm(stations):
    series = merged_df[station_id]["v"]
    valid_hours = [i for i, x in enumerate(series.isna()) if not x]
    
    # cut off any NaNs at the beginning or end
    series = series.iloc[valid_hours[0]:valid_hours[-1]]

    # calculate hours with NaNs
    invalid_hours = [i for i, x in enumerate(series.isna()) if x]

    # count up consecutiveness
    consecutive_nans = countlist(invalid_hours)
    consecutive_nan_dict[station_id] = consecutive_nans

In [ ]:
consecutive_nan_dict

In [ ]:
# save this dict to the /data drive somewhere. not sure if we will use it, but it's small to store and time-consuming to reproduce
import pickle
with open("/data/surge/consecutive-nans-for-tidegauges.pickle", "wb") as f:
    pickle.dump(consecutive_nan_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
all_nan_lengths = []
for key, val in consecutive_nan_dict.items():
    all_nan_lengths.extend(val)

plt.hist(all_nan_lengths, bins=100, range=(0, 7500))
plt.title("length of outages across all eastern US tide gauges")
plt.xlabel("hours")
plt.ylabel("count")
plt.show()

In [ ]:
np.median(all_nan_lengths)

## Figure out what % of each gauge's history it has been operating successfully

### Test with one gauge first

In [ ]:
id = "8410140"

# clip record to operating period
series = merged_df.loc[:, idx[id, "v"]]
not_nans = np.argwhere(~np.array(series.isna().tolist()))
start, end = not_nans[0], not_nans[-1]

In [ ]:
start, end = start[0], end[0]

In [ ]:
series = series.iloc[start:end]
series

In [ ]:
fraction_valid = np.sum(~np.array(series.isna().tolist())) / len(series)
fraction_valid

### Repeat for all gauges

In [ ]:
stations = [x[0] for x in merged_df.columns.tolist()]
stations = list(set(stations))

valid_records = {}
for station_id in tqdm(stations):
    series = merged_df.loc[:, idx[station_id, "v"]]
    not_nans = np.argwhere(~np.array(series.isna().tolist()))
    start, end = not_nans[0], not_nans[-1]
    start, end = start[0], end[0]
    series = series.iloc[start:end]
    fraction_valid = np.sum(~np.array(series.isna().tolist())) / len(series)
    valid_records[station_id] = fraction_valid
    print(fraction_valid)

In [ ]:
# make a histogram of the valid length for each gauge
fracs = []
for key, val in valid_records.items():
    fracs.append(val)

plt.hist(fracs, bins=20)
plt.xlabel("Fraction of record that is valid data")
plt.ylabel("Count")
plt.title("Valid fraction of tide gauge records (variable record lengths)")
plt.show()

In [ ]:
np.median(fracs)

In [ ]:
1- 0.957410081230533

In [ ]:
# Save this history since it is quite time-consuming to generate
import pickle
with open("/data/surge/fraction-of-record-valid.pickle", "wb") as f:
    pickle.dump(fracs, f, protocol=pickle.HIGHEST_PROTOCOL)

## How easily can we detect surge?

### Attempt a simple quantile approach on a single gauge first
Get the 95th percentile of the gauge height (MSL). Find areas in the record where the gauge's height exceedes this level. Plot some time before and after.

NB: ideally we may want to detrend sea level rise before doing this, or else calculate quantiles on a rolling basis. The 95% of gauge height will be very different in 1920 vs 2020 since sea level has risen about 30 cm! 

In [ ]:
station_id = "8410140"
series = merged_df.loc[:, idx[station_id, "v"]]
# get a list of the valid values
series_dropnans = series.dropna()

In [ ]:
# get the 99% quantile
high_level = series_dropnans.quantile(0.999)
print(high_level)

In [ ]:
series[(series > high_level) & ~series.isna()]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16,6))
series.loc["2023-01-20":"2023-01-26"].plot(ax=ax)
ax.hlines(y=high_level, xmin="2023-01-20", xmax="2023-01-27", color="gray", linestyle="dashed")

In [ ]:
# highest ever level
np.max(series_dropnans)

In [ ]:
series[(series > 4.35) & ~series.isna()]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16,6))
series.loc["2020-04-05":"2020-04-15"].plot(ax=ax, label="water level")
ax.hlines(y=4.35, xmin="2020-04-05", xmax="2020-04-16", color="gray", linestyle="dashed", label="highest recorded value")
ax.set_title(f"Water level in Eastport, ME (gauge {station_id})")
ax.set_ylabel("Hourly water level")
plt.legend()
plt.show()

In [ ]:
# any record of flooding or storms during that date?
# get the station name
noaa_coops.Station(station_id).name

Nothing obvious in the news about coastal flooding during this period...